<a href="https://colab.research.google.com/github/Firdaous2002/Experiments_thesis/blob/main/addition_of_discrete_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.9 MB/s eta 0:00:00


In [3]:
import torch
import torch_geometric
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

# Load the Cora dataset
dataset = Planetoid(root='data/Planetoid', name='Cora')

# Access the dataset (Cora has only one graph)
data = dataset[0]  # Single graph in Cora
print(data)

# Inspect the data
print("Number of nodes:", data.num_nodes)
print("Number of edges:", data.num_edges)
print("Number of features per node:", data.num_node_features)
print("Number of classes:", dataset.num_classes)


Processing...


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Number of features per node: 1433
Number of classes: 7


Done!


In [10]:
# Number of nodes to subset
num_subset_nodes = 10

# Subset the feature matrix (first 10 nodes)
subset_V_x = dataset[0].x[:num_subset_nodes]

# Subset the adjacency matrix (first 10 rows and columns)
subset_adj = dataset[0].edge_index

# Initialize a smaller perturbation matrix (for the subset)
P_subset = torch.randn(num_subset_nodes, dataset.num_node_features)


In [11]:
from typing import Tuple
from torch import Tensor

#I tried to modify forward_prediction and forward methods in order to be able to have the ability to add features as well, not only deactivate them. Of course I only retuned the modified feature matrix to see if it modified the features as i wanted.

In [12]:
def forward_prediction(V_x: Tensor, P_x: Tensor) -> Tuple[Tensor, Tensor, Tensor]:
    """
    Thresholded forward pass for prediction.

    Args:
        V_x (Tensor): Original node feature matrix (num_nodes, num_features).

    Returns:
        Tuple[Tensor, Tensor, Tensor]:
            - Model's output.
            - Perturbed feature matrix.
            - Raw perturbation matrix.
    """
    # Apply tanh to P_x to create real-valued perturbations
    real_perturbations = torch.tanh(P_x)

    # Threshold perturbations to {-1, 0, +1} for discrete features
    thresholded_perturbations = torch.where(
        real_perturbations > 0.5, 1.0,  # Add feature
        torch.where(real_perturbations < -0.5, -1.0, 0.0)  # Remove or retain
    )
    features_add = False
    if not features_add:
        # Apply thresholded perturbations (add or remove features)
        V_pert = torch.clamp(V_x + thresholded_perturbations, 0, 1)
    else:
        # Use perturbations directly (already thresholded)
        V_pert = torch.clamp(thresholded_perturbations, 0, 1)


    return V_pert, real_perturbations



In [13]:
# Call the function on the subset
V_pert, real_perturbations = forward_prediction(subset_V_x, P_subset)

# Print outputs for inspection
print("Perturbed Features (Subset):\n", V_pert)
print("Real-Valued Perturbations (Subset):\n", real_perturbations)
print(subset_V_x)


Perturbed Features (Subset):
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 1.],
        [0., 0., 1.,  ..., 1., 0., 1.]])
Real-Valued Perturbations (Subset):
 tensor([[-0.0742, -0.2305, -0.0023,  ..., -0.6793, -0.5133, -0.6491],
        [ 0.8900,  0.3226, -0.7839,  ..., -0.9449, -0.5405, -0.9916],
        [-0.4408, -0.2273, -0.4385,  ..., -0.9444, -0.9412, -0.4204],
        ...,
        [ 0.1563, -0.4874, -0.1869,  ..., -0.4505,  0.0691,  0.4846],
        [ 0.7161,  0.7757,  0.8231,  ...,  0.4804, -0.1561,  0.8709],
        [ 0.1738, -0.8230,  0.7765,  ...,  0.6527, -0.5391,  0.7489]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0

In [9]:
# Get the features of the first node
node_features = V_pert[0]

# Find indices of ones
indices_of_ones = torch.nonzero(node_features == 1).squeeze().tolist()

# Find indices of zeros
indices_of_zeros = torch.nonzero(node_features == 0).squeeze().tolist()

print("Indices with value 1:", indices_of_ones)
print("Indices with value 0:", indices_of_zeros)

Indices with value 1: [0, 1, 2, 4, 8, 13, 19, 21, 24, 26, 30, 31, 32, 34, 36, 39, 40, 43, 44, 46, 48, 50, 52, 57, 58, 67, 69, 73, 74, 86, 87, 88, 90, 93, 96, 97, 98, 114, 115, 116, 120, 125, 126, 130, 131, 132, 133, 135, 137, 144, 146, 147, 148, 154, 156, 159, 161, 163, 164, 165, 175, 176, 181, 187, 188, 189, 194, 198, 199, 201, 204, 210, 211, 217, 222, 223, 225, 226, 230, 233, 234, 235, 238, 243, 244, 246, 247, 249, 251, 253, 259, 262, 264, 266, 271, 273, 276, 277, 280, 281, 286, 289, 294, 297, 299, 300, 301, 303, 315, 316, 318, 320, 322, 327, 330, 334, 336, 337, 339, 340, 341, 342, 351, 352, 358, 361, 362, 374, 375, 380, 382, 383, 386, 395, 396, 399, 402, 406, 409, 412, 414, 415, 418, 422, 427, 436, 437, 444, 446, 447, 451, 453, 461, 462, 465, 467, 471, 472, 473, 474, 476, 478, 483, 489, 499, 502, 505, 506, 508, 513, 516, 520, 525, 526, 534, 539, 544, 546, 547, 549, 551, 552, 558, 573, 574, 575, 577, 578, 582, 585, 591, 592, 596, 602, 625, 634, 635, 637, 638, 639, 640, 641, 644, 647,

In [10]:
# Get the features of the first node
node_features = subset_V_x[0]

# Find indices of ones
indices_of_ones = torch.nonzero(node_features == 1).squeeze().tolist()

# Find indices of zeros
indices_of_zeros = torch.nonzero(node_features == 0).squeeze().tolist()

print("Indices with value 1:", indices_of_ones)
print("Indices with value 0:", indices_of_zeros)

Indices with value 1: [19, 81, 146, 315, 774, 877, 1194, 1247, 1274]
Indices with value 0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 2

In [11]:
def forward(subset_V_x: Tensor, subset_adj: Tensor, P_subset: Tensor) -> Tensor:
    """
    Differentiable forward pass for training on a subset of the data.

    Args:
        subset_V_x (Tensor): Subset node feature matrix (num_subset_nodes, num_features).
        subset_adj (Tensor): Subset adjacency matrix (edges for the subset).
        P_subset (Tensor): Perturbation matrix for the subset (num_subset_nodes, num_features).

    Returns:
        Tensor: Model's output after applying differentiable perturbations.
    """
    # Apply tanh for differentiable perturbations
    real_perturbations = torch.tanh(P_subset)
    features_add = False
    if not features_add:
        # Apply perturbations to the input features
        V_pert = torch.clamp(subset_V_x + real_perturbations, 0, 1)
    else:
        # Use perturbations directly
        V_pert = torch.clamp(real_perturbations, 0, 1)

    # Pass perturbed features through the model
    return V_pert, real_perturbations


In [12]:
V_pert, real_perturbations = forward(subset_V_x, subset_adj, P_subset)

print("Output for the subset:", V_pert)
print("Output for the subset:", real_perturbations)

Output for the subset: tensor([[0.7915, 0.9488, 0.5231,  ..., 0.0000, 0.3382, 0.0000],
        [0.8915, 0.4844, 0.7070,  ..., 0.0000, 0.0000, 0.0309],
        [0.9794, 0.2781, 0.7277,  ..., 0.1018, 0.0000, 0.2707],
        ...,
        [0.9683, 0.1273, 0.0000,  ..., 0.0000, 0.9812, 0.5251],
        [0.0000, 0.3425, 0.0000,  ..., 0.7057, 0.0000, 0.3198],
        [0.0000, 0.1821, 0.8987,  ..., 0.4518, 0.3862, 0.0000]])
Output for the subset: tensor([[ 0.7915,  0.9488,  0.5231,  ..., -0.4209,  0.3382, -0.8648],
        [ 0.8915,  0.4844,  0.7070,  ..., -0.2592, -0.3851,  0.0309],
        [ 0.9794,  0.2781,  0.7277,  ...,  0.1018, -0.5680,  0.2707],
        ...,
        [ 0.9683,  0.1273, -0.6233,  ..., -0.6649,  0.9812,  0.5251],
        [-0.0535,  0.3425, -0.8753,  ...,  0.7057, -0.9189,  0.3198],
        [-0.9218,  0.1821,  0.8987,  ...,  0.4518,  0.3862, -0.5050]])
